In [86]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'


In [57]:
train_ori = pd.read_pickle(PATH + 'train_bo0.pkl')
test_ori = pd.read_pickle(PATH + 'test_bo0.pkl')
pred_ori = pd.read_csv(PATH + 'submission/blend1.csv')

In [58]:
def large_new(df1, df2, prediction, threshold):
    print('begin cut large')
    pred = prediction.copy()
    train_df, test_df = df1.copy(), df2.copy()
    print('old shapes,' ,train_df.shape,test_df.shape)
    half_pred = pred[pred['TARGET'] > threshold]
    index_half = half_pred.index
    print('length of half',len(half_pred))
    test_half = test_df.iloc[index_half]
    test_half['TARGET'] = 1

    # drop index_half for test_df, add train
    test_df.drop(index_half, inplace = True)
    train_df = pd.concat([train_df,test_half])
    print('new shapes,',train_df.shape,test_df.shape)
    return train_df, test_df, half_pred

In [65]:
pred_ori.describe()

,SK_ID_CURR,TARGET
count,48744.000000,48744.000000
mean,277796.676350,0.071744
std,103169.547296,0.084132
min,100001.000000,0.002001
25%,188557.750000,0.021216
50%,277549.000000,0.041974
75%,367555.500000,0.087919
max,456250.000000,0.778711


In [71]:
train, test, cut_large = large_new(train_ori, test_ori, pred_ori, 0.6)

begin cut large
old shapes, (307511, 2395) (48744, 2394)
length of half 91


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


new shapes, (307602, 2395) (48653, 2394)


In [61]:
def small_new(df1, df2, prediction, threshold):
    print('begin cut small')
    pred = prediction.copy()
    train_df, test_df = df1.copy(), df2.copy()
    print('old shapes,',test_df.shape,train_df.shape)
    small_pred = pred[pred['TARGET'] < threshold]
    index_small = small_pred.index
    print('length of small',len(small_pred))
    test_small = test_df.iloc[index_small]
    test_small['TARGET'] = 0

    # drop index_small for test_df, add train
    test_df.drop(index_small, inplace = True)
    train_df = pd.concat([train_df,test_small])
    print('new shapes,',test_df.shape,train_df.shape)
    return train_df, test_df, small_pred

In [69]:
# train, test, cut_small = small_new(train_ori, test_ori, pred_ori, 0.005)

In [72]:
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'
print('begin cv')
target = train['TARGET']
test_df = test.copy()
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train.columns if x not in ignore_cols]
train = train[features]
test = test[features]

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,

                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 4000, 'verbose': 0 ,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1,'num_threads': 12,'feature_fraction': 0.2,
#               'scale_pos_weight':1}
# lgbm_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.05,
#     'num_iteration': 4000,
#     'num_threads': 8,

#     'num_leaves': int(round(44.368535336628419)),
#     'feature_fraction': 0.28231763168020257,
#     'bagging_fraction': 0.94901525271474951,
#     'max_depth': int(round(8.0430115561596267)),
#     'lambda_l1': 0.30680079516647751,
#     'lambda_l2': 0.079128660903201031,
#     'min_split_gain': 0.054005067457890979,
#     'min_child_weight': 98.172643147364937}
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.02,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.15,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 15.172643147364937}


cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
#     small_len = out_df[out_df['TARGET']< threshold1]
#     large_len = out_df[out_df['TARGET']> threshold2]
#     print('length of small and large,',len(small_len),len(large_len))
print((out_df['TARGET']>0.5).sum(),'more than half')


begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.761096 + 0.00310528
[200]	cv_agg's auc: 0.773691 + 0.00260612
[300]	cv_agg's auc: 0.782541 + 0.00293995
[400]	cv_agg's auc: 0.787528 + 0.00310356
[500]	cv_agg's auc: 0.790788 + 0.00315946
[600]	cv_agg's auc: 0.79275 + 0.00314467
[700]	cv_agg's auc: 0.794057 + 0.00311753
[800]	cv_agg's auc: 0.794783 + 0.00308738
[900]	cv_agg's auc: 0.795314 + 0.00303433
[1000]	cv_agg's auc: 0.795683 + 0.00315248
[1100]	cv_agg's auc: 0.795943 + 0.0032424
[1200]	cv_agg's auc: 0.796202 + 0.00333095
[1300]	cv_agg's auc: 0.796239 + 0.00331112
[1400]	cv_agg's auc: 0.796264 + 0.00336876
[1500]	cv_agg's auc: 0.7963 + 0.00332573
[1600]	cv_agg's auc: 0.79618 + 0.00334676
Optimum boost rounds = 1486
Best CV result = 0.7963404625747211


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


139 more than half


In [73]:
out_df.shape

(48653, 2)

In [74]:
cut_large.shape

(91, 2)

In [77]:
test = pd.read_csv(PATH + 'application_test.csv')
test = test[['SK_ID_CURR']]
pred = pd.concat([cut_large,out_df], axis = 0)
test = test.merge(pred, on = 'SK_ID_CURR', how = 'left')

In [79]:
test.head()
test[test.TARGET > 0.5]

,SK_ID_CURR,TARGET
634,104410,0.701353
998,106854,0.634645
1537,110993,0.500587
1555,111146,0.502004
1610,111452,0.654957
1829,112917,0.511019
1935,113627,0.719967
2035,114283,0.725944
2513,117520,0.646886
2565,117920,0.597811


In [81]:
pred_803 = pd.read_csv(PATH + 'submission/blend1.csv')
x = test.TARGET - pred_803.TARGET
x.describe()

count    48744.000000
mean        -0.000467
std          0.019563
min         -0.231371
25%         -0.004784
50%         -0.000285
75%          0.004543
max          0.269011
Name: TARGET, dtype: float64

In [83]:
abs(x).sum()
test.to_csv(PATH + 'submission/blend_large.csv', index = False)

# Analysis for pred

In [87]:
pred_1 = pd.read_csv(PATH+'submission/06_29_0.csv')
pred_2 = pd.read_csv(PATH +'test_2/pred_2.csv')
pred_3 = pd.read_csv(PATH +'test_2/pred_3.csv')


In [89]:
blend1 = pd.read_csv(PATH + 'submission/blend1.csv')
blend3 = pred_1.copy()
blend3.TARGET = pred_1.TARGET * 0.5 + pred_3.TARGET *0.5

In [93]:
x = blend1 - blend3
abs(x.TARGET).sum()
blend3.to_csv(PATH + 'submission/blend4.csv', index = False)

In [95]:
blend4 = pred_1.copy()
blend4.TARGET = pred_1.TARGET * 0.7 + pred_3.TARGET *0.3
blend4.to_csv(PATH + 'submission/blend5.csv', index = False)

In [46]:
x = pred_1 - pred_2
y = pred_1 - pred_3
z = pred_2 - pred_3
for i in [x,y,z]:
    print(i.describe())
    print(sum(abs(i['TARGET'])))

       SK_ID_CURR        TARGET
count     48744.0  48744.000000
mean          0.0     -0.007282
std           0.0      0.068466
min           0.0     -0.838735
25%           0.0     -0.008297
50%           0.0     -0.000458
75%           0.0      0.007144
max           0.0      0.374950
1100.093830680137
       SK_ID_CURR        TARGET
count     48744.0  48744.000000
mean          0.0     -0.001022
std           0.0      0.029821
min           0.0     -0.317782
25%           0.0     -0.008087
50%           0.0     -0.000381
75%           0.0      0.007339
max           0.0      0.374950
816.9059598365895
       SK_ID_CURR        TARGET
count     48744.0  4.874400e+04
mean          0.0  6.260306e-03
std           0.0  5.690829e-02
min           0.0 -5.551115e-17
25%           0.0  0.000000e+00
50%           0.0  0.000000e+00
75%           0.0  0.000000e+00
max           0.0  5.997406e-01
305.1523598621816


In [5]:
print(pred_1.shape, pred_2.shape, pred_3.shape)

(48744, 2) (48744, 2) (48744, 2)


In [6]:
x = (pred_1.TARGET >0.5) & (pred_2.TARGET >0.5) & (pred_3.TARGET >0.5)

In [7]:
x.sum()

116

In [8]:
pred_1.loc[x]

,SK_ID_CURR,TARGET
634,104410,0.657005
998,106854,0.600391
1610,111452,0.685340
1935,113627,0.723145
2035,114283,0.661640
2513,117520,0.617668
2585,118060,0.525313
2593,118127,0.538932
2764,119362,0.541435
2830,119821,0.546086


In [19]:
train_ori = pd.read_pickle(PATH + 'train_bo0.pkl')
test_ori = pd.read_pickle(PATH + 'test_bo0.pkl')


In [20]:
tmp = test_ori.copy()
test_large = tmp.loc[x]
test_large['TARGET'] = 1
print('original,', train_ori.shape, test_ori.shape)
print(test_large.shape)

original, (307511, 2395) (48744, 2394)
(116, 2395)


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
# test = test_ori.drop(test_large, axis = 0)
train = pd.concat([train_ori, test_large], axis = 0)
test = test_ori.loc[~x]
print(train.shape, test.shape)

(307627, 2395) (48628, 2394)


In [22]:
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'
print('begin cv')
target = train['TARGET']
test_df = test.copy()
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train.columns if x not in ignore_cols]
train = train[features]
test = test[features]

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,

                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 4000, 'verbose': 0 ,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1,'num_threads': 12,'feature_fraction': 0.2,
#               'scale_pos_weight':1}
# lgbm_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.05,
#     'num_iteration': 4000,
#     'num_threads': 8,

#     'num_leaves': int(round(44.368535336628419)),
#     'feature_fraction': 0.28231763168020257,
#     'bagging_fraction': 0.94901525271474951,
#     'max_depth': int(round(8.0430115561596267)),
#     'lambda_l1': 0.30680079516647751,
#     'lambda_l2': 0.079128660903201031,
#     'min_split_gain': 0.054005067457890979,
#     'min_child_weight': 98.172643147364937}
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.02,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.15,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 15.172643147364937}


cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
#     small_len = out_df[out_df['TARGET']< threshold1]
#     large_len = out_df[out_df['TARGET']> threshold2]
#     print('length of small and large,',len(small_len),len(large_len))
print((out_df['TARGET']>0.5).sum(),'more than half')

begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.761302 + 0.00298085
[200]	cv_agg's auc: 0.773986 + 0.0028527
[300]	cv_agg's auc: 0.782747 + 0.00276872
[400]	cv_agg's auc: 0.787774 + 0.00257664
[500]	cv_agg's auc: 0.790973 + 0.00233752
[600]	cv_agg's auc: 0.792932 + 0.00214634
[700]	cv_agg's auc: 0.794195 + 0.0020962
[800]	cv_agg's auc: 0.794974 + 0.00207049
[900]	cv_agg's auc: 0.795535 + 0.00202317
[1000]	cv_agg's auc: 0.795967 + 0.00202895
[1100]	cv_agg's auc: 0.796189 + 0.00192961
[1200]	cv_agg's auc: 0.796331 + 0.0020462
[1300]	cv_agg's auc: 0.796436 + 0.00202969
[1400]	cv_agg's auc: 0.796473 + 0.00203762
[1500]	cv_agg's auc: 0.796432 + 0.00200176
Optimum boost rounds = 1393
Best CV result = 0.7964847374342525


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


122 more than half


In [23]:
part1 = pred_1.loc[x]
part2 = out_df

In [26]:
print(part1.shape, part2.shape)
p = pd.concat([part1, part2], axis = 0)

(116, 2) (48628, 2)


In [25]:
test = test_ori[['SK_ID_CURR']]

In [27]:
test = test.merge(p, on = 'SK_ID_CURR', how = 'left')
test.head()

,SK_ID_CURR,TARGET
0,100001,0.042861
1,100005,0.123038
2,100013,0.028493
3,100028,0.026717
4,100038,0.156317


In [28]:
(test.TARGET>0.5).sum()

238

In [29]:
test.to_csv(PATH + 'submission/80_large.csv', index = False)

In [85]:
test = test_ori[['SK_ID_CURR']]
blend1 = pred_1['TARGET'] * 0.5 + pred_2['TARGET'] *0.1 + pred_3['TARGET'] * 0.4
test['TARGET'] = blend1



/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [51]:
(test['TARGET']>0.5).sum()

316

In [52]:
test.to_csv(PATH + 'submission/blend1.csv', index = False)